In [1]:
import torch
import torchvision
import torchvision.datasets as datasets
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Parameters
learning_rate = 0.005
momentum = 0.0
num_epochs = 100
batch_size = 32
dropout_rate = 0.2

#Comment this out for L2 regularization
regularization = 'l1'
l1 = 0.9

# To transform to tensor
transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# Dataset for training, validation and test sets as tensors
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms)
mnist_trainset, mnist_validationset = torch.utils.data.random_split(mnist_trainset, [50000, 10000])

num_training_samples = mnist_trainset.__len__()
l1 = (l1 * 1.0) / num_training_samples

# Data loader for train, test and validation sets
trainloader = torch.utils.data.DataLoader(mnist_trainset, batch_size=batch_size, num_workers=2, shuffle=True)
testloader = torch.utils.data.DataLoader(mnist_testset, batch_size=batch_size, num_workers=2, shuffle=True)
validationloader = torch.utils.data.DataLoader(mnist_validationset, batch_size=batch_size, num_workers=2, shuffle=True)

# Compute accuracy
def Accuracy(dataLoader):
    total_samples, score = 0, 0
    for i, data in enumerate(dataLoader):
        with torch.no_grad():
            inputs, labels = data
            outputs = model(inputs)
            outputs = torch.argmax(outputs, dim=1)
            correct = sum(outputs == labels).data.to('cpu').numpy()

            total_samples = total_samples + batch_size
            score = score + correct

    accuracy = score * 1.0 / total_samples
    return accuracy

class Net(nn.Module):

    # Defining the network
    def __init__(self):
        super(Net, self).__init__()
        
        # 28*28 input image channel, 10 output channels
        self.input_size = 28*28
        self.hidden1_size = 100
        self.hidden2_size = 100
        self.output_size = 10
        
        # Defining the layers and activation functions
        self.layer1 = nn.Linear(self.input_size, self.hidden1_size)
        self.layer2 = nn.Linear(self.hidden1_size, self.hidden2_size)
        self.layer3 = nn.Linear(self.hidden2_size, self.output_size)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.softmax = nn.Softmax()
        
    # Forward propagation
    def forward(self, x):
        x = x.view(-1, self.input_size) # Reshaping to a vector
        
        # Activation functions
        x = F.relu(self.layer1(x))
        x = self.dropout(x)
        x = F.relu(self.layer2(x))
        x = self.dropout(x)
        x = self.softmax(self.layer3(x))
        return x

# Getting our model
model = Net()

# Defining optimizer and loss function
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum) # SGD
loss_func = nn.CrossEntropyLoss() # Cross Entropy

# Starting Training
for epoch in range (0, num_epochs):
    epoch_loss = 0
    for i, data in enumerate(trainloader):
        x, label = data
        inputs, labels = data
        output = model(inputs)

        model.zero_grad()
        loss = loss_func(output, labels)
        
        # For L1 regularization, SGD already performs L2 by default
        if regularization == "l1":
            for param in model.parameters():
                loss = loss + l1 * torch.sum(torch.abs(param)).data.to('cpu').numpy()
        
        loss.backward()
        optimizer.step()
        
        epoch_loss = epoch_loss + loss
        
    print("Epoch {}. Loss = {}".format(epoch, "%.2f" % epoch_loss))

    # Validation accuracy every 10 epochs
    if (epoch) % 10 == 0:
        print("Epoch {}. Validation Accuracy = {}".format(epoch, "%.2f" % Accuracy(validationloader)))

# Test set accuracy
print("Test Accuracy = {}".format("%.2f" % Accuracy(testloader)))

C:\Users\Adg\Anaconda3\lib\site-packages\ipykernel_launcher.py:79: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0. Loss = 1827.54
Epoch 0. Validation Accuracy = 0.15
Epoch 1. Loss = 1826.49
Epoch 2. Loss = 1825.16
Epoch 3. Loss = 1823.47
Epoch 4. Loss = 1820.96
Epoch 5. Loss = 1816.70
Epoch 6. Loss = 1806.27
Epoch 7. Loss = 1783.19
Epoch 8. Loss = 1760.75
Epoch 9. Loss = 1731.91
Epoch 10. Loss = 1691.90
Epoch 10. Validation Accuracy = 0.39
Epoch 11. Loss = 1658.61
Epoch 12. Loss = 1631.56
Epoch 13. Loss = 1591.09
Epoch 14. Loss = 1548.08
Epoch 15. Loss = 1517.14
Epoch 16. Loss = 1496.29
Epoch 17. Loss = 1482.25
Epoch 18. Loss = 1471.49
Epoch 19. Loss = 1456.56
Epoch 20. Loss = 1445.18
Epoch 20. Validation Accuracy = 0.69
Epoch 21. Loss = 1435.51
Epoch 22. Loss = 1428.15
Epoch 23. Loss = 1421.05
Epoch 24. Loss = 1415.90
Epoch 25. Loss = 1411.50
Epoch 26. Loss = 1408.66
Epoch 27. Loss = 1404.59
Epoch 28. Loss = 1402.40
Epoch 29. Loss = 1398.89
Epoch 30. Loss = 1396.92
Epoch 30. Validation Accuracy = 0.72
Epoch 31. Loss = 1395.13
Epoch 32. Loss = 1392.93
Epoch 33. Loss = 1391.94
Epoch 34. Los

KeyboardInterrupt: 